In [31]:
import pandas as pd
import numpy as np
import utils as ut # esta librería tiene funciones para poder obtener un procesamiento del <T,H>
import spacy
import mutual_info as mi
import time
from scipy.stats import wasserstein_distance
import sys

import conceptnet_lite
conceptnet_lite.connect("../OPENAI/data/conceptnet.db")
from conceptnet_lite import Label, edges_for, edges_between


import textacy
from textacy import extract
from functools import partial

In [32]:
## esta función revisa hiperonimia, sinonimia entre otras.
def compatibilidad_semantica(word_t,word_h):
    #Nos quedamos con el lemma para identificar las relaciones blue{ blue ,ADJ}​
    wt=str(word_t).split("{")[1].split(",")[0]
    wh=str(word_h).split("{")[1].split(",")[0]
    try:
        concepts_wt = Label.get(text=wt, language='en').concepts
        concepts_wh = Label.get(text=wh, language='en').concepts
        #Sinonimia
        for e in edges_between(concepts_wt, concepts_wh, two_way=True,relation="synonym"):
            print(wh," synonym ",wt)
            return True,1,"synonym"
        #Antonimos
        for e in edges_between(concepts_wt, concepts_wh, two_way=True,relation="antonym"):
            print(wt," antonym ",wh)
            return True,2,"antonym"
        for e in edges_between(concepts_wt, concepts_wh, two_way=True,relation="distinct_from"):
            print(wh," distinct from ",wt)
            return True,2,"distinct_from"
        #Hiperonimia o Hiponimia
        for e in edges_between(concepts_wt, concepts_wh, two_way=True,relation="is_a"):
            if wt== e.start.text:
                print(wt," is_a ",wh)
                return True,1,"is_a"
            else:
                print(wt," is_a (ant) ",wh)
                return True,2,"is_a_c"
        for e in edges_between(concepts_wt, concepts_wh, two_way=True,relation="manner_of"):
            if wt== e.start.text:
                print(wt," manner_of ",wh)
                return True,1,"manner_of"
            else:
                print(wt," manner_of (ant) ",wh)
                return True,2,"manner_of_c"
        for e in edges_between(concepts_wh,concepts_wt, two_way=True,relation="has_a"):
            if wt== e.start.text:
                print(wh," has_a ",wt)
                return True,1,"has_a"
            else:
                print(wh,"has_a (ant)",wt)
                return True,2,"has_a_c"
        for e in edges_between(concepts_wt,concepts_wh, two_way=True,relation="derived_from"):
            if wt== e.start.text:
                print(wt," derivado_from ",wh)
                return True,1,"derived_from"
            else:
                print(wh," derivado_from (ant)",wt)
                return True,2,"derived_from_c"
        for e in edges_between(concepts_wt,concepts_wh, two_way=True,relation="used_for"):
            if wt== e.start.text:
                print(wt," used_for ",wh)
                return True,1,"used_for"
            else:
                print(wh,"used_for (ant)",wt)
                return True,2,"used_for_c"
        for e in edges_between(concepts_wt,concepts_wh, two_way=True,relation="has_property"):
            if wt== e.start.text:
                print(wt," HasProperty ",wh)
                return True,1,"HasProperty"
            else:
                print(wh," HasProperty ",wt)
                return True,2,"hasProperty_c"
        # Inflexión
        for e in edges_between(concepts_wt, concepts_wh, two_way=True,relation="form_of"):
            print(wt," form_of ",wh)
            return True,1,"form_of"
        #Implicación
        for e in edges_between(concepts_wt,concepts_wh, two_way=True,relation="causes"):
            if wt== e.start.text:
                print(wt," causes ",wh)
                return True,1,"Causes"
            else:
                print(wh," causes ",wt)
                return True,2,"Causes_c"
        for e in edges_between(concepts_wt,concepts_wh, two_way=True,relation="has_prerequisite"):
            if wt== e.start.text:
                print(wt," has_prerequisite ",wh)
                return True,1,"has_prerequisite"
            else:
                print(wh," has_prerequisite ",wt)
                return True,2,"has_prerequisite_c"
        for e in edges_between(concepts_wt,concepts_wh, two_way=True,relation="has_subevent"):
            if wt== e.start.text:
                print(wt," has_subevent ",wh)
                return True,1,"has_subevent"
            else:
                print(wh," has_subevent ",wt)
                return True,2,"has_subevent_c"
        for e in edges_between(concepts_wt,concepts_wh, two_way=True,relation="has_first_subevent"):
            if wt== e.start.text:
                print(wt," has_first_subevent ",wh)
                return True,1,"has_first_subevent"
            else:
                print(wh," has_first_subevent ",wt)
                return True,2,"has_first_subevent_c"
            
        #relaciones conceptuales
        for e in edges_between(concepts_wt,concepts_wh, two_way=True,relation="related_to"):
            print(wt," related_to ",wh)
            return True,3,"related_to"
        for e in edges_between(concepts_wt,concepts_wh, two_way=True,relation="similar_to"):
            print(wt," similar_to ",wh)
            return True,3,"similar_to"
            
    except:
        pass
    return False,0,""

In [33]:
def obtener_distancia(texto_v,hipotesis_v,texto_t,texto_h,b_col,b_index):
    lista_l=[]
    for i in range(len(texto_t)):
        lista=[]
        for j in range(len(texto_h)):
            lista.append(np.linalg.norm(texto_v[i] - hipotesis_v[j]))#*wasserstein_distance(texto_2[i],hipotesis_2[j]))
        lista_l.append(lista)
    df_distEuc=pd.DataFrame(lista_l,index=texto_t,columns=texto_h)
    df_distEuc=df_distEuc.drop(b_col[1:],axis=1)
    df_distEuc=df_distEuc.drop(b_index[1:],axis=0)
    return df_distEuc

In [34]:
def wasserstein_mutual_inf(texto_v,hipotesis_v,texto_t,texto_h):  
    lista_l=[]
    lista_muinfor=[]   
    for i in range(len(texto_t)):
        lista=[]
        lista_mu=[]
        for j in range(len(texto_h)):
            lista.append(wasserstein_distance(texto_v[i],hipotesis_v[j]))
            lista_mu.append(mi.mutual_information_2d(np.array(texto_v[i]),np.array(hipotesis_v[j])))
        lista_l.append(lista)
        lista_muinfor.append(lista_mu)
    DFmearth=pd.DataFrame(lista_l,index=texto_t,columns=texto_h)
    DFmutual_inf=pd.DataFrame(lista_muinfor,index=texto_t,columns=texto_h)
    return DFmearth,DFmutual_inf

In [35]:
def entropia(X):
    """Devuelve el valor de entropia de una muestra de datos""" 
    probs = [np.mean(X == valor) for valor in set(X)]
    return round(sum(-p * np.log2(p) for p in probs), 3)

In [36]:
def bi_grams(texto1,texto2):
    gramsT=set()
    gramsH=set()
    palabras = texto1
    for i in range(len(palabras)-1):
        gramsT.add((palabras[i],palabras[i+1]))
    palabras = texto2
    for i in range(len(palabras)-1):
        gramsH.add((palabras[i],palabras[i+1]))
    if len(gramsH)!=0:
        return len(gramsT.intersection(gramsH))/len(gramsH)
    else:
        return 0

In [37]:
def tri_grams(texto1,texto2):
    gramsT=set()
    gramsH=set()
    palabras = texto1
    for i in range(len(palabras)-2):
        gramsT.add((palabras[i],palabras[i+1],palabras[i+2]))
    palabras = texto2
    for i in range(len(palabras)-2):
        gramsH.add((palabras[i],palabras[i+1],palabras[i+2]))
    if len(gramsH)!=0:
        return len(gramsT.intersection(gramsH))/len(gramsH)
    else:
        return 0

In [38]:
def cuatri_grams(texto1,texto2):
    gramsT=set()
    gramsH=set()
    palabras = texto1
    for i in range(len(palabras)-3):
        gramsT.add((palabras[i],palabras[i+1],palabras[i+2],palabras[i+3]))
    palabras = texto2
    for i in range(len(palabras)-3):
        gramsH.add((palabras[i],palabras[i+1],palabras[i+2],palabras[i+3]))
    if len(gramsH)!=0:
        return len(gramsT.intersection(gramsH))/len(gramsH)
    else:
        return 0

In [39]:
def get_grams_entities(texto):
    finales=[]
    doc = textacy.make_spacy_doc(texto, lang="en_core_web_md")
    terms = list(extract.terms(doc,
    ngs=partial(extract.ngrams, n=2, include_pos={"NOUN", "ADJ", "VERB","ADV","PART"}),
    ents=partial(extract.entities, include_types={"PERSON", "ORG", "GPE", "LOC"}),
    dedupe=False))
    a = list(extract.subject_verb_object_triples(doc))
    for e in terms:
        finales.append(str(e))
    for e in a:
        trip = ""
        for e1 in e[0]:
            trip += str(e1) + " "
        for e1 in e[1]:
            trip += str(e1) + " "
        for e1 in e[2]:
            trip += str(e1) + " "
        finales.append(trip)
    return finales

In [40]:
def bag_of_synonyms(word):
    sinonimos=set()
    try:
        for e in edges_for(Label.get(text=word, language='en').concepts, same_language=True):
            if e.relation.name == "synonym":
                if word== e.start.text:
                    sinonimos.add(e.end.text)
                elif word== e.end.text:
                    sinonimos.add(e.start.text)
    except:
        pass
    return sinonimos

In [41]:
def bag_of_antonyms(word):
    antonimos=set()
    try:
        for e in edges_for(Label.get(text=word, language='en').concepts, same_language=True):
            if e.relation.name in ["antonym","distinc_from"]:
                if word== e.start.text:
                    antonimos.add(e.end.text)
                elif word== e.end.text:
                    antonimos.add(e.start.text)
    except:
        pass
    return antonimos

In [42]:
relaciones=["is_a","manner_of","has_a","derived_from","used_for","has_property","form_of","causes","has_prerequisite","has_subevent","has_first_subevent"]

In [43]:
def bag_of_hyperonyms(word):
    hiperonimos=set()
    try:
        for e in edges_for(Label.get(text=word, language='en').concepts, same_language=True):
            if e.relation.name in relaciones:
                if word== e.start.text:
                    hiperonimos.add(e.end.text)
    except:
        pass
    return hiperonimos

In [44]:
def bag_of_hyponyms(word):
    hiponimos=set()
    try:
        for e in edges_for(Label.get(text=word, language='en').concepts, same_language=True):
            if e.relation.name in relaciones:
                if word== e.end.text:
                    hiponimos.add(e.start.text)
    except:
        pass
    return hiponimos

In [45]:
nlp = spacy.load("en_core_web_md") # modelo de nlp

#ut.load_vectors_in_lang(nlp,"../OPENAI/data/glove.840B.300d.txt") # carga de vectores en nlp.wv
ut.load_vectors_in_lang(nlp,"./data/numberbatch-en-17.04b.txt") # carga de vectores en nlp.wv

In [46]:
prueba=pd.read_csv("../OPENAI/data/RTE3/DEV_1.csv")
#prueba=pd.read_csv("../OPENAI/data/"+sys.argv[1])

In [47]:
textos = prueba["sentence1"].to_list()       # almacenamiento en listas
hipotesis = prueba["sentence2"].to_list()

In [48]:
# lista de listas para dataframe
new_data = {'sumas' : [], 'distancias' : [], 'entropia_total' : [],'entropias' : [],'mutinf' : [], 
            'mearts' : [], 'max_info' : [], 'similitud_faltantes' : [], 'list_comp' : [],
            'list_incomp' : [], 'list_rel_con' : [], 'list_M' : [], 'list_m' : [],
            'list_T' : [], 'list_relaciones' : [], 'listas_malign' : [], 'listas_malignf' : [],
            'list_bigram' : [], 'list_trigram' : [], 'list_cuatrigram' : [], 'diferencias':[],
            'bi_ent_trip_t':[],'bi_ent_trip_h':[],'bi_ent_trip_rel':[], 'clases' : []}


In [49]:
inicio = time.time()
for i in range(len(textos)):
#for i in range(7,8):
    print(i)

    t_clean_m=ut.get_words(textos[i],nlp,pos_to_remove=['PUNCT'], normed=True,lemmatize=False)
    h_clean_m=ut.get_words(hipotesis[i],nlp,pos_to_remove=['PUNCT'], normed=True,lemmatize=False)

    t_clean_m=ut.reform_sentence2(t_clean_m,nlp)
    h_clean_m=ut.reform_sentence2(h_clean_m,nlp)

    print("reformado:",t_clean_m,h_clean_m)

    t_clean=ut.get_words_rep(t_clean_m,nlp,pos_to_remove=[""],normed=True,lemmatize=False)
    h_clean=ut.get_words_rep(h_clean_m,nlp,pos_to_remove=[""],normed=True,lemmatize=False)
    print("limpios",t_clean,h_clean)
    print(len(t_clean),len(h_clean))

    t_vectors=ut.get_matrix_rep(t_clean_m,nlp, pos_to_remove=[""],normed=False,lemmatize=False)
    h_vectors=ut.get_matrix_rep(h_clean_m,nlp, pos_to_remove=[""],normed=False,lemmatize=False)
    t_vectors_n=ut.get_matrix_rep(t_clean_m,nlp, pos_to_remove=[""],normed=True,lemmatize=False)
    h_vectors_n=ut.get_matrix_rep(h_clean_m,nlp, pos_to_remove=[""],normed=True,lemmatize=False)

    #print(t_vectors.shape)
    # ngrams en 
    t_ngrams=get_grams_entities(textos[i])
    h_ngrams=get_grams_entities(hipotesis[i])
    print(t_ngrams,h_ngrams)
    new_data['bi_ent_trip_t'].append(t_ngrams)
    new_data['bi_ent_trip_h'].append(h_ngrams)

    lenh=len(set(h_ngrams))
    lent=len(set(t_ngrams))

    if lenh!=0 and lent!=0:
        new_data['bi_ent_trip_rel'].append(len(set(t_ngrams).intersection(set(h_ngrams)))/lenh)
    else:
        new_data['bi_ent_trip_rel'].append(0)

    # Obtencion de matriz de alineamiento, matriz de move earth y mutual information
    ma=np.dot(t_vectors_n,h_vectors_n.T)
    #print(len(t_vectors_n),len(h_vectors_n),len(t_clean),len(h_clean))
    m_earth,m_mi=wasserstein_mutual_inf(t_vectors_n,h_vectors_n,t_clean,h_clean)

    ma=pd.DataFrame(ma,index=t_clean,columns=h_clean)
    pru_eba=ma
    # Calculamos la entropia inicial de la matriz de distancias coseno sobre tokens de T y H
    new_data['entropia_total'].append(entropia(ma.round(1).values.flatten())) 
    
    new_data['list_bigram'].append(bi_grams(t_clean,h_clean))
    new_data['list_trigram'].append(tri_grams(t_clean,h_clean))
    new_data['list_cuatrigram'].append(cuatri_grams(t_clean,h_clean))


    ###### BORRADO DE COSAS QUE NO OCUPO, SOLO NOS QUEDAMOS CON INFORMACIÓN DE TIPOS DE PALABRA: NOUN, VERB, ADJ Y ADV
    # TAMBIÉN OMITIMOS EL VERBO BE DEBIDO A QUE POR LO REGULAR SE UTILIZA COMO AUXILIAR Y ES UN VERBO COPULATIVO
    # sirve para construir la llamada predicación nominal del sujeto de una oración: 
    # #el sujeto se une con este verbo a un complemento obligatorio llamado atributo que por lo general determina 
    # alguna propiedad, estado o equivalencia del mismo, por ejemplo: "Este plato es bueno". "Juan está casado".
    
    b_col=[0]
    
    new_data['list_T'].append(ma.shape[0])
    new_data['list_M'].append(ma.shape[1])
    new_data['listas_malign'].append(ma)
    #print(ma.index,ma.columns)
    col=ma.columns
    borrar=[]
    indexes=ma.index
    for c in col:
        if "{null," in str(c) or "{be,VERB" in str(c) or (",NUM"  not in str(c) and "not," not in str(c) and "PRON" not in str(c) and "NOUN" not in str(c) and "VERB" not in str(c) and "ADJ" not in str(c) and "ADV" not in str(c)):
            borrar.append(c)        
        elif str(c) in indexes:
            borrar.append(c)        
    #borrar_i=[]
    #for index in indexes:
    #    if "{null," in str(index) or "{be,VERB" in str(index) or (",NUM"  not in str(index) and "not," not in str(index) and "PRON" not in str(index) and "NOUN" not in str(index) and "VERB" not in str(index) and "ADJ" not in str(index) and "ADV" not in str(index)):
    #        borrar_i.append(index) 
    #    elif str(index) in col:
    #        borrar_i.append(index) 
    
    ma=ma.drop(borrar,axis=1)
    #ma=ma.drop(borrar_i,axis=0)
    m_earth=m_earth.drop(borrar,axis=1)
    #m_earth=m_earth.drop(borrar_i,axis=0)
    m_mi=m_mi.drop(borrar,axis=1)
    #m_mi=m_mi.drop(borrar_i,axis=0)
    
    b_col.extend(borrar)
    c_compatibilidad=0
    c_incompatibilidad=0
    c_rel_concep=0

    # ELIMINAMOS INFORMACIÓN DONDE SE CORRESPONDAN EN LEMMAS, TENGA UN PRODUCTO IGUAL A 1 Y SEAN IGUALES LOS INDICES Y COLUMNAS
    # SI EL VALOR ES IGUAL A 1 SIGNIFICA QUE ES LA MISMA PALABRA, O SON SINONIMOS
    borrar=[]
    borrar_i=[]
    col=ma.columns
    for index,strings in ma.iterrows():
        lema_i=str(index).split("{")[1].split(",")[0]
        for c in col:
            if index==c:
                borrar_i.append(index)
                borrar.append(c)
            # if strings[c]>=1:
            #     borrar_i.append(index)
            #     borrar.append(c)
            lema_c=str(c).split("{")[1].split(",")[0]
            if lema_i == lema_c:
                borrar_i.append(index)
                borrar.append(c)
    ma=ma.drop(borrar,axis=1)
    #ma=ma.drop(borrar_i,axis=0)
    m_earth=m_earth.drop(borrar,axis=1)
    #m_earth=m_earth.drop(borrar_i,axis=0)
    m_mi=m_mi.drop(borrar,axis=1)
    #m_mi=m_mi.drop(borrar_i,axis=0)
    b_col.extend(borrar)
    #primera vuelta ---------------------------------------------------------------------------------
    # #PARA REVISAR SI EXISTEN RELACIONES DE SIMILITUD SEMÁNTICA A TRAVÉS DEL USO DE CONCEPNET
    
    n_index = ma.shape[0]
    n_columns = ma.shape[1]
    pasada=0

    parejas=""
    print(ma.index,ma.columns)
    print(ma)
    while n_columns>0 and pasada<5:
        borrar=[]
        a = ma.idxmax().values
        b = ma.columns
        for j in range(len(a)):
            print(a[j],b[j])
            #match,grupo,relacion = compatibilidad_semantica(a[j],b[j])
            wt_s=str(a[j]).split("{")[1].split(",")[0]
            wh_s=str(b[j]).split("{")[1].split(",")[0]
            sin_t=bag_of_synonyms(wt_s)
            ant_t=bag_of_antonyms(wt_s)
            hipe_t=bag_of_hyperonyms(wt_s)
            hipo_t=bag_of_hyponyms(wt_s)
            sin_h=bag_of_synonyms(wh_s)
            ant_h=bag_of_antonyms(wh_s)
            hipe_h=bag_of_hyperonyms(wh_s)
            hipo_h=bag_of_hyponyms(wh_s)
            print("sinonimos de T",sin_t)
            print("antonimos de T",ant_t)
            print("Hiperonimos de T",hipe_t)
            print("Hiponimos de T",hipo_t)
            print("Sinonimos de H",sin_h)
            print("Antonimos de H",ant_h)
            print("Hiperonimos de H",hipe_h)
            print("Hiponimos de H",hipo_h)
            #COMPATIBILIDAD SEMÁNTICA
            if len(sin_t.intersection(sin_h))>0 or wt_s in sin_h or wh_s in sin_t:
                print(a[j],b[j],sin_t.intersection(sin_h),"sinonimos sinonimos")
                c_compatibilidad+=1
                borrar.append(b[j])
            if len(ant_t.intersection(ant_h))>0:
                print(a[j],b[j],ant_t.intersection(ant_h),"antonimos de antonimos")
                c_compatibilidad+=1
                borrar.append(b[j])
            if len(hipe_t.intersection(sin_h))>0:
                print(a[j],b[j],hipe_t.intersection(sin_h),"hiperonimo - sinonimos")
                c_compatibilidad+=1
                borrar.append(b[j])
            if len(hipo_t.intersection(hipo_h))>0:
                print(a[j],b[j],hipo_t.intersection(hipo_h),"hiponimo - hiponimos")
                c_compatibilidad+=1
                borrar.append(b[j])
            if len(sin_t.intersection(hipo_h))>0:
                print(a[j],b[j],sin_t.intersection(ant_t),"sinonimos hiponimos")
                c_compatibilidad+=1
                borrar.append(b[j])
            if wt_s in hipo_h:
                print(a[j],b[j],wt_s,"generalidad")
                c_compatibilidad+=1
                borrar.append(b[j])
            #INCOMPATIBILIDAD SEMÁNTICA
            if wt_s in hipe_h:
                print(a[j],b[j],wt_s,"especifididad")
                c_incompatibilidad+=1
                borrar.append(b[j])
            if len(sin_t.intersection(ant_h))>0:
                print(a[j],b[j],sin_t.intersection(ant_t),"sinonimos antonimos")
                c_incompatibilidad+=1
                borrar.append(b[j])
            if len(sin_t.intersection(hipe_h))>0:
                print(a[j],b[j],sin_t.intersection(ant_t),"sinonimos hiperonimos")
                c_incompatibilidad+=1
                borrar.append(b[j])
            if len(ant_t.intersection(sin_h))>0 or wh_s in ant_t:
                print(a[j],b[j],ant_t.intersection(sin_h),wh_s,ant_t,"antonimos sinonimos")
                c_incompatibilidad+=1
                borrar.append(b[j])
            if len(ant_t.intersection(hipe_h))>0:
                print(a[j],b[j],ant_t.intersection(hipe_h),"antonimos hiperonimos")
                c_incompatibilidad+=1
                borrar.append(b[j])
            if len(ant_t.intersection(hipo_h))>0:
                print(a[j],b[j],ant_t.intersection(hipo_h),"antonimos hiponimos")
                c_incompatibilidad+=1
                borrar.append(b[j])
            if len(hipe_t.intersection(ant_h))>0:
                print(a[j],b[j],hipe_t.intersection(ant_h),"hiperonimos antonimos")
                c_incompatibilidad+=1
                borrar.append(b[j])
            if len(hipe_t.intersection(hipe_h))>0:
                print(a[j],b[j],hipe_t.intersection(hipe_h),"hiperonimos hiperonimos | cohiponimia")
                c_incompatibilidad+=1
                borrar.append(b[j])
            if len(hipo_t.intersection(sin_h))>0:
                print(a[j],b[j],hipo_t.intersection(sin_h),"hiponimos sinonimos")
                c_incompatibilidad+=1
                borrar.append(b[j])
            if len(hipo_t.intersection(ant_h))>0:
                print(a[j],b[j],hipo_t.intersection(ant_h),"hiponimos antonimos")
                c_incompatibilidad+=1
                borrar.append(b[j])
            if len(hipo_t.intersection(hipe_h))>0:
                print(a[j],b[j],hipo_t.intersection(hipe_h),"hiponimos hiperonimos")
                c_incompatibilidad+=1
                borrar.append(b[j])
            # if match and grupo==1:
            #     borrar.append(b[j])
            #     c_compatibilidad+=1
            #     parejas = parejas + a[j] + " - " + relacion + " - "+ b[j]+ " | "
            # elif match and grupo==2:
            #     borrar.append(b[j])
            #     c_incompatibilidad+=1
            #     parejas = parejas + a[j] + " - " + relacion + " - "+ b[j]+ " | "
            # elif match and grupo==3:
            #     borrar.append(b[j])
            #     c_rel_concep+=1
            #     parejas = parejas + a[j] + " - " + relacion + " - "+ b[j]+ " | "
        #ma = ma.drop(borrar_i)
        ma = ma.drop(borrar,axis=1)
        m_earth = m_earth.drop(borrar,axis=1)
        #m_earth = m_earth.drop(borrar_i)
        m_mi = m_mi.drop(borrar,axis=1)
        #m_mi = m_mi.drop(borrar_i)
        n_columns = ma.shape[1]
        pasada+=1
        b_col.extend(borrar)
    b_index=[0]
    #   ALMACENAMIENTO DE TODA LA INFORMACIÓN PROCESADA DE CARACTERÍSTICAS
    m_distancia = obtener_distancia(t_vectors,h_vectors,t_clean,h_clean,b_col,b_index)
    
    new_data['distancias'].append(m_distancia.max().sum()) #cambie de maximas a sumas
    #m_earth=m_earth*m_distancia
    if ma.shape[1]==0:
        new_data['entropias'].append(0)
        new_data['max_info'].append(0)
        new_data['sumas'].append(0)
        new_data['mearts'].append(0)
        new_data['mutinf'].append(0)
        new_data['diferencias'].append(0)
    else:
        new_data['entropias'].append(entropia(ma.round(1).values.flatten()))
        new_data['max_info'].append(ma.max().sum()/(ma.shape[1]))# 
        new_data['sumas'].append(ma.sum().sum()/(ma.shape[1]))# 
        new_data['mearts'].append(m_earth.min().sum()/(ma.shape[1]))# 
        new_data['mutinf'].append(m_mi.max().sum()/(ma.shape[1]))# 
        new_data['diferencias'].append(len(ma.columns)/len(ma.index))

    new_data['list_comp'].append(c_compatibilidad)
    new_data['list_incomp'].append(c_incompatibilidad)
    new_data['list_rel_con'].append(c_rel_concep)
    new_data['list_relaciones'].append(parejas)
    new_data['listas_malignf'].append(ma)
    new_data['list_m'].append(ma.shape[1])

    st=""
    sh=""
    for t__1 in ma.index:
        st=st+" "+str(t__1).split("{")[0]
    for t__2 in ma.columns:
        sh=sh+" "+str(t__2).split("{")[0]
    doc1 = nlp(st)
    doc2 = nlp(sh)

    if sh!="" and st!="":
        new_data['similitud_faltantes'].append(doc1.similarity(doc2))
    elif st!="" and sh=="":
        new_data['similitud_faltantes'].append(1)
    else:
        new_data['similitud_faltantes'].append(0)
    new_data['clases'].append(prueba.at[i,"gold_label"])
fin = time.time()
print("Tiempo que se llevo:",round(fin-inicio,2)," segundos")

0
reformado: the sale was made to pay yukos us$ 27 5 billion tax tax_bill bill yuganskneftegaz was originally sold for us$ 9 4 billion to a little little_known known company baikalfinansgroup which was later bought by the russian state owned oil oil_company company rosneft  baikalfinansgroup was sold to rosneft 
limpios ['the{the,DET}', 'sale{sale,NOUN}', 'was{be,AUX}', 'made{make,VERB}', 'to{to,PART}', 'pay{pay,VERB}', 'yukos{yukos,PROPN}', 'us${us$,SYM}', '27{27,NUM}', '5{5,NUM}', 'billion{billion,NUM}', 'tax{tax,NOUN}', 'tax_bill{tax_bill,NOUN}', 'bill{bill,PROPN}', 'yuganskneftegaz{yuganskneftegaz,PROPN}', 'was{be,AUX}', 'originally{originally,ADV}', 'sold{sell,VERB}', 'for{for,ADP}', 'us${us$,SYM}', '9{9,NUM}', '4{4,NUM}', 'billion{billion,NUM}', 'to{to,ADP}', 'a{a,DET}', 'little{little,ADJ}', 'little_known{little_known,ADJ}', 'known{know,VERB}', 'company{company,NOUN}', 'baikalfinansgroup{baikalfinansgroup,NOUN}', 'which{which,PRON}', 'was{be,AUX}', 'later{later,ADV}', 'bought{bu

/tmp/ipykernel_4178170/4097985338.py:283: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  new_data['similitud_faltantes'].append(doc1.similarity(doc2))


Index(['cauhtemoc{cauhtemoc,PROPN}', 'cardenas{cardenas,PROPN}',
       'said{say,VERB}', 'during{during,ADP}', 'a{a,DET}', 'news{news,NOUN}',
       'news_conference{news_conference,NUM}', 'conference{conference,NOUN}',
       'on{on,ADP}', '7{7,NUM}', 'june{june,PROPN}', 'that{that,SCONJ}',
       'the{the,DET}', 'visit{visit,NOUN}', 'to{to,ADP}',
       'mexico{mexico,PROPN}', 'by{by,ADP}', 'salvadoran{salvadoran,PROPN}',
       'president{president,PROPN}', 'alfredo{alfredo,PROPN}',
       'cristiani{cristiani,PROPN}', 'is{be,AUX}', 'a{a,DET}',
       'visit{visit,NOUN}', 'by{by,ADP}', 'a{a,DET}',
       'repressive{repressive,ADJ}', 'ruler{ruler,NOUN}', 'who{who,PRON}',
       'oppresses{oppress,VERB}', 'a{a,DET}', 'large{large,ADJ}',
       'sector{sector,NOUN}', 'of{of,ADP}', 'of_people{of_people,ADJ}',
       'people{people,NOUN}'],
      dtype='object') Index([], dtype='object')
Empty DataFrame
Columns: []
Index: [cauhtemoc{cauhtemoc,PROPN}, cardenas{cardenas,PROPN}, said{say,

In [66]:
pru_eba.idxmax()

30{30,NUM}                                  30{30,NUM}
die{die,VERB}                     injured{injure,VERB}
die_in{die_in,NUM}                            a{a,DET}
in{in,ADP}                                  in{in,ADP}
a{a,DET}                                      a{a,DET}
bus{bus,NOUN}                            bus{bus,NOUN}
collision{collision,NOUN}    collision{collision,NOUN}
in{in,ADP}                                  in{in,ADP}
uganda{uganda,PROPN}              uganda{uganda,PROPN}
dtype: object

In [83]:
rank=pru_eba.rank(ascending = False,method = "min")
rank["die{die,VERB}"].idxmin()

'injured{injure,VERB}'

In [84]:
rank.drop(rank["die{die,VERB}"].idxmin())

,"30{30,NUM}","die{die,VERB}","die_in{die_in,NUM}","in{in,ADP}","a{a,DET}","bus{bus,NOUN}","collision{collision,NOUN}","in{in,ADP}","uganda{uganda,PROPN}"
"a{a,DET}",21.0,16.0,1.0,15.0,1.0,12.0,13.0,15.0,13.0
"bus{bus,NOUN}",11.0,12.0,1.0,22.0,20.0,1.0,16.0,22.0,20.0
"collision{collision,NOUN}",4.0,5.0,1.0,9.0,6.0,3.0,1.0,9.0,17.0
"with{with,ADP}",17.0,15.0,1.0,4.0,4.0,18.0,8.0,4.0,10.0
"a{a,DET}",21.0,16.0,1.0,15.0,1.0,12.0,13.0,15.0,13.0
"truck{truck,NOUN}",15.0,7.0,1.0,21.0,16.0,2.0,7.0,21.0,22.0
"truck_in{truck_in,NOUN}",18.0,9.0,1.0,23.0,21.0,21.0,23.0,23.0,23.0
"in{in,ADP}",6.0,19.0,1.0,1.0,12.0,16.0,11.0,1.0,6.0
"uganda{uganda,PROPN}",12.0,8.0,1.0,13.0,18.0,10.0,22.0,13.0,1.0
"has{have,AUX}",24.0,24.0,1.0,11.0,9.0,24.0,17.0,11.0,8.0


In [69]:
pru_eba.columns

Index(['30{30,NUM}', 'die{die,VERB}', 'die_in{die_in,NUM}', 'in{in,ADP}',
       'a{a,DET}', 'bus{bus,NOUN}', 'collision{collision,NOUN}', 'in{in,ADP}',
       'uganda{uganda,PROPN}'],
      dtype='object')

In [44]:
pru_eba.max()

30{30,NUM}                   1.000000
die{die,VERB}                0.231184
die_in{die_in,NUM}           0.000000
in{in,ADP}                   1.000000
a{a,DET}                     1.000000
bus{bus,NOUN}                1.000000
collision{collision,NOUN}    1.000000
in{in,ADP}                   1.000000
uganda{uganda,PROPN}         1.000000
dtype: float32

In [60]:
pru_eba.idxmax()[0]

/tmp/ipykernel_4160020/94606754.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pru_eba.idxmax()[0]


'30{30,NUM}'

In [50]:
pru_eba.drop(pru_eba.idxmax()).idxmax()

30{30,NUM}                                  21{21,NUM}
die{die,VERB}                         left{leave,VERB}
die_in{die_in,NUM}                      with{with,ADP}
in{in,ADP}                              and{and,CCONJ}
a{a,DET}                                with{with,ADP}
bus{bus,NOUN}                        truck{truck,NOUN}
collision{collision,NOUN}    fatalities{fatality,NOUN}
in{in,ADP}                              and{and,CCONJ}
uganda{uganda,PROPN}                    and{and,CCONJ}
dtype: object

In [54]:
psss = pru_eba.drop(pru_eba.idxmax())
psss 

,"30{30,NUM}","die{die,VERB}","die_in{die_in,NUM}","in{in,ADP}","a{a,DET}","bus{bus,NOUN}","collision{collision,NOUN}","in{in,ADP}","uganda{uganda,PROPN}"
"with{with,ADP}",0.013580,-0.067191,0.0,0.376365,0.267174,0.013460,0.291459,0.376365,0.167343
"truck{truck,NOUN}",0.027706,0.020926,0.0,0.040093,0.118597,0.534601,0.341907,0.040093,0.037011
"truck_in{truck_in,NOUN}",0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
"has{have,AUX}",-0.062913,-0.147312,0.0,0.207544,0.213641,-0.051734,0.173849,0.207544,0.171623
"resulted{result,VERB}",0.100101,0.023570,0.0,0.373594,0.265766,0.011810,0.405688,0.373594,0.201831
"in_least{in_least,NOUN}",0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
"at{at,ADP}",0.068674,-0.140882,0.0,0.307933,0.027787,0.194151,0.109625,0.307933,0.073182
"at_least{at_least,NUM}",0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
"least{least,ADV}",0.150007,0.176606,0.0,0.105114,0.204411,0.089219,0.273718,0.105114,0.125089
"fatalities{fatality,NOUN}",0.109789,0.156589,0.0,0.283841,0.175171,0.105443,0.561846,0.283841,0.222457


In [56]:
psss.drop(psss.idxmax()).idxmax()

30{30,NUM}                          least{least,ADV}
die{die,VERB}                       least{least,ADV}
die_in{die_in,NUM}           truck_in{truck_in,NOUN}
in{in,ADP}                     resulted{result,VERB}
a{a,DET}                       resulted{result,VERB}
bus{bus,NOUN}                             at{at,ADP}
collision{collision,NOUN}       further{further,ADJ}
in{in,ADP}                     resulted{result,VERB}
uganda{uganda,PROPN}            further{further,ADJ}
dtype: object

In [ ]:
#clases=prueba["gold_label"].values

#temp1 =[sumas,distancias,entropia_total,entropias,mutinf,mearts,max_info,similitud_faltantes,list_comp,list_incomp,list_rel_con,list_M,list_m,list_T,list_relaciones,listas_malign,listas_malignf,list_bigram,list_trigram,list_cuatrigram,clases]
#df_resultados = pd.DataFrame(temp1,columns=["suma","distancias","entropia_total","entropias","mutual_info","m_earth","max_info_p","sim_faltantes","Compatibilidad","Incompatibilidad","Rel_conceptuales","Shape Origin","Shape Finish","Total T","Match","Ma","Maf","bigram","trigram","cuatrigram","CLASS"])
df_resultados = pd.DataFrame(new_data)
#df_resultados.to_csv("salida/final/"+sys.argv[1]+".csv",index=False)
#df_resultados.to_pickle("salida/prueba/"+sys.argv[1]+"_.pickle")



In [22]:
textos

["The sale was made to pay Yukos' US$ 27.5 billion tax bill, Yuganskneftegaz was originally sold for US$ 9.4 billion to a little known company Baikalfinansgroup which was later bought by the Russian state-owned oil company Rosneft .",
 "The sale was made to pay Yukos' US$ 27.5 billion tax bill, Yuganskneftegaz was originally sold for US$9.4 billion to a little known company Baikalfinansgroup which was later bought by the Russian state-owned oil company Rosneft .",
 'Loraine besides participating in Broadway\'s Dreamgirls, also participated in the Off-Broadway production of "Does A Tiger Have A Necktie". In 1999, Loraine went to London, United Kingdom. There she participated in the production of "RENT" where she was cast as "Mimi" the understudy.',
 '"The Extra Girl" (1923) is a story of a small-town girl, Sue Graham (played by Mabel Normand) who comes to Hollywood to be in the pictures. This Mabel Normand vehicle, produced by Mack Sennett, followed earlier films about the film industry